In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import xgboost as xgb

train = pd.read_csv('../input/playground-series-s4e5/train.csv',index_col="id")
test = pd.read_csv('../input/playground-series-s4e5/test.csv',index_col="id")

In [3]:
target = "FloodProbability"
X = train.drop(target, axis=1)
y = train[target]

randomState = 41
nSplits = 10
nRepeats = 10
kfold = RepeatedStratifiedKFold(n_splits=nSplits, n_repeats=nRepeats, random_state=randomState)

cv = []
for train_index, test_index in kfold.split(X,y.astype(str)):
    cv.append((train_index, test_index))

X_new =  np.sort(X.values, axis=1)
X_new = np.c_[X_new, X_new.sum(axis=1)]

scores = cross_val_score(
    xgb.XGBRegressor(n_jobs=4, random_state=0),
    X_new, y, cv = cv, n_jobs=1
)

print(F'{scores.mean():.5f} ± {scores.std():.5f}')  

0.86908 ± 0.00056
